In [23]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:

import datetime
import read_cloudsat as rc
import pandas as pd
import ftplib
import numpy as np
import os

In [2]:
CLOUDSAT_LINK = "ftp.cloudsat.cira.colostate.edu"
def get_cloudsat_files(start: dt.datetime, end: dt.datetime, dest: str, username: str, password: str):
    """Gets hdf files from the CloudSat FTP server based on date

    Args:
        start (dt.datetime): Start date for data.
        end (dt.datetime): End date for data.
        dest (str): Destination folder we want to download to.
        username (str): CloudSat username
        password (str): CloudSat password
    """

    dt_range = pd.date_range(st_dt, ed_dt, freq="1H")
    cs_fnames = []
    date = []

    # Get files for that day
    ftp = ftplib.FTP(CLOUDSAT_LINK, user=username, passwd=password)
    ftp.set_pasv(True)
    ftp.login(user=username, passwd=password)

    for idt in dt_range:
        # Making base file names we want to get
        doy = idt.timetuple().tm_yday
        target_path = "~/2B-GEOPROF.P1_R05/{}/{:03d}".format(idt.year, doy)
        
        #2012297175936_34524_CS_2B-GEOPROF_GRANULE_P_R04_E06_ATL-18L
        fstr = '%s*_R05_*.hdf'%idt.strftime('%Y%j%H')
        ftp.cwd(target_path)
        file_names = ftp.nlst(fstr)
        for file_name in file_names:
            local_filename = os.path.join(dest, file_name)
            if os.path.exists(local_filename):
                print("Skipping " + file_name)
                continue
            file = open(local_filename, 'wb')
            ftp.retrbinary('RETR '+ file_name, file.write)
            print("Downloaded " + file_name)
            file.close()
    ftp.quit()
    return rc.read_cloudsat(dest, st_dt, ed_dt)

In [4]:
st = datetime.datetime(2015, 1, 1, 0, 0, 0)   
ed = datetime.datetime(2015, 1, 1, 10, 0, 0)
dat = get_cloudsat_files(st, ed, "./data/", "hirungolwe", "537Luna537!")

./data/2015001031028_46165_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf 201501010300
./data/2015001044921_46166_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf 201501010400
./data/2015001062814_46167_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf 201501010600
./data/2015001080707_46168_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf 201501010800
./data/2015001094600_46169_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf 201501010900


In [5]:
dat

<xarray.Dataset>
Dimensions:     (obs_id: 185407, chan_elem: 125, channel: 1)
Coordinates:
  * obs_id      (obs_id) int64 0 1 2 3 4 ... 185402 185403 185404 185405 185406
  * chan_elem   (chan_elem) int64 1 2 3 4 5 6 7 ... 119 120 121 122 123 124 125
  * channel     (channel) int64 1
Data variables:
    nelem_chan  (channel) float64 125.0
    lat         (obs_id) float64 -0.009408 -0.01908 ... 0.01796 0.008287
    lon         (obs_id) float64 -20.96 -20.96 -20.96 ... -144.6 -144.6 -144.6
    height      (obs_id, chan_elem) int16 -9999 -9999 -9999 ... -9999 -9999
    time        (obs_id) datetime64[ns] 2015-01-01T03:10:28 ... 2015-01-01T11...
    obs         (obs_id, chan_elem) float64 nan nan nan nan ... nan nan nan nan

In [28]:
a = dat.lat[:]
b = np.tile(a, (125, 1)).T
print(b.shape)

(185407, 125)


In [104]:
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np
import datetime as dt

def plot_side_2d(data, start:dt.datetime, end: dt.datetime, title:str,
                      save_path:str="./2d_images/"):
    """Plots a contour plot of Latitude x Altitude x Reflectivity, given CloudSat data.

    Args:
        data (netCDF4 Dataset): 2A.GPM.DPRX.V8 data from GPM.
        start (int): Range of indices the function should plot graphs for.
        end (int): Range of indices the function should plot graphs for.
        title (str): Title for plot.
        save_path (str): Where the screenshot should be saved to.
    """
    my_cmap = [(57/255, 78/255, 157/255), (0, 159/255, 60/255), (248/255, 244/255, 0),(1, 0, 0), (1, 1, 1)]
    my_cmap = colors.LinearSegmentedColormap.from_list("Reflectivity", my_cmap, N=26)

    all_dt = data.time.values
    start = np.datetime64(start)
    end = np.datetime64(end)
    good_ind = np.where((all_dt >= start) & (all_dt <= end))[0]

    lat = data.lat[good_ind].values
    lat = np.tile(lat, (125, 1))
    alt = data.height[good_ind, :].values.T / 1000.0
    obs = data.obs[good_ind, :].values.T
    obs[obs <= -24] = np.nan

    vmin, vmax = -28.5, 47.5
    plt.figure(figsize=(20, 12))
    plt.contourf(lat, alt, obs, vmin=vmin, vmax=vmax, cmap=my_cmap)
    plt.title(title)
    plt.xlabel("Latitude, deg")
    plt.ylabel("Altitude, km")
    plt.colorbar(label="Reflectivity")
    plt.ylim((0, 17))
    plt.savefig(save_path + "/img.png", facecolor='white', transparent=False)

In [ ]:
start = dt.datetime(2008, 9, 13, 1, 20, 7)

end = dt.datetime(2008, 9, 13, 13, 25, 53)
plot_side_2d(dat, start, end, "title")

In [101]:
np.datetime64(start)

numpy.datetime64('2008-09-13T01:20:07.000000')

In [58]:
# 'lat', 'lon', 'height', 'time', 'obs'
lower: datetime.datetime = datetime.datetime(2008, 9, 13, 0, 0, 0)
upper: datetime.datetime = datetime.datetime(2008, 9, 13, 12, 0, 0)
low_lat, up_lat = 28.6, 31.4
dat = rc.read_cloudsat("./data/", lower, upper)
lat = dat.lat
lat = lat[(lat >= low_lat) & (lat <= up_lat)]
time = dat["time"].isel(obs_id=lat["obs_id"]).values.T
# alt = dat.height.isel(obs_id=lat.obs_id).values.T / 1000.0
# obs = dat["obs"].isel(obs_id=lat["obs_id"]).values.T
# lat = lat.values
# lat = np.tile(lat, (125, 1))
# obs[obs <= -24] = np.nan



print("Height is", alt.shape)
print("Reflect is", obs.shape)
print("Latitude is", lat.shape)


./data/2008257002243_12650_CS_2B-GEOPROF_GRANULE_P1_R05_E02_F00.hdf 200809130000
./data/2008257020136_12651_CS_2B-GEOPROF_GRANULE_P1_R05_E02_F00.hdf 200809130200
./data/2008257034029_12652_CS_2B-GEOPROF_GRANULE_P1_R05_E02_F00.hdf 200809130300
./data/2008257051922_12653_CS_2B-GEOPROF_GRANULE_P1_R05_E02_F00.hdf 200809130500
./data/2008257065815_12654_CS_2B-GEOPROF_GRANULE_P1_R05_E02_F00.hdf 200809130600
./data/2008257083709_12655_CS_2B-GEOPROF_GRANULE_P1_R05_E02_F00.hdf 200809130800
./data/2008257101602_12656_CS_2B-GEOPROF_GRANULE_P1_R05_E02_F00.hdf 200809131000
./data/2008257115455_12657_CS_2B-GEOPROF_GRANULE_P1_R05_E02_F00.hdf 200809131100
Height is (125, 296654)
Reflect is (125, 296654)
Latitude is (4659,)


In [68]:
print(min(time))


2008-09-13T01:20:07.799072000


datetime.datetime(2008, 9, 13, 13, 25, 53)

In [132]:
import matplotlib.colors as colors

frames = 10
step = 50

my_cmap = [(57/255, 78/255, 157/255), (0, 159/255, 60/255), (248/255, 244/255, 0),(1, 0, 0), (1, 1, 1)]
my_cmap = colors.LinearSegmentedColormap.from_list("Reflectivity", my_cmap, N=26)
vmin, vmax = -28.5, 47.5

#plt.xlim((low_lat, up_lat))
text = "Figure 7 Bottom"
last_s, s = 0, 1000

for s in range(s, lat.shape[1], step):
    plt.contourf(lat[:, last_s:s], alt[:, last_s:s], obs[:, last_s:s], vmin=vmin, vmax=vmax, cmap=my_cmap)
    plt.title(text)
    plt.xlabel("Latitude, deg")
    plt.ylabel("Altitude, km")
    plt.colorbar(label="Reflectivity")
    plt.ylim((0, 17))
    plt.savefig("./2d_images/test{}.png".format(last_s), facecolor='white', transparent=False)
    plt.clf()
    last_s += 50

<Figure size 432x288 with 0 Axes>

In [ ]:
import matplotlib.animation as animation
#plt.rcParams['animation.ffmpeg_path'] = r"C:\some_path\ffmpeg.exe"   # if necessary

fig = plt.figure()
ax = plt.axes(ylim=(0, 17), xlabel='x', ylabel='y')
frames = 50
step = lat.shape[1] // frames
last_s, s = 0, 1000

cont = plt.contourf(lat[:, last_s:s], alt[:, last_s:s], obs[:, last_s:s], vmin=vmin, vmax=vmax, cmap=my_cmap, levels=26)
plt.colorbar()

# animation function
def animate(i):
    global cont
    global last_s
    global s
    x = lat[:, last_s:s]
    y = alt[:, last_s:s]
    z = obs[:, last_s:s]
    last_s += step
    s += step
    plt.clf()
    cont = plt.contourf(x, y, z, vmin=vmin, vmax=vmax, cmap=my_cmap, levels=26)
    plt.title('t = %i' % (i))
    return cont

anim = animation.FuncAnimation(fig, animate, frames=frames, repeat=False)
anim.save('animation.mp4', writer=animation.FFMpegWriter())

In [1]:
import GPMPy as gm
import datetime as dt
import xarray as xr
from os import listdir
import drpy.core.gpmdpr as drp
from multiprocessing.dummy import Pool as ThreadPool

def load_one_gpm(filename):
    print(filename)
    tmp = drp.GPMDPR(filename=filename)
    bad_ones = ['flagSurfaceSnow','binBBTop','binBBBottom','flagPrecip','typePrecip','phaseNearSurface','precipRateNearSurface',
            'nearsurfaceKu','nearsurfaceKa','epsilon','MSKa_c','NSKu','MSKa','R','Dm_dpr','Nw_dpr']
    return tmp.xrds.drop_vars(bad_ones)

def load_multi_gpm(files, multi_thread=False):
    if multi_thread:
        pool = ThreadPool(len(files))
        all_ds = pool.map(load_one_gpm, files)
        pool.close()
        pool.join()
    else:
        all_ds = []
        for filename in files:
            tmp = load_one_gpm(filename)
            all_ds.append(tmp)
    return all_ds

# 2m55s w/ no thread
# 1m56s w/ n threads!!
data_path = "data/"
files = listdir(data_path)

files = [data_path + f for f in files if ".HDF5" in f]
all_ds = load_multi_gpm(files, True)
combine_ds = xr.concat(all_ds, dim="along_track")

data/2A.GPM.DPRX.V8-20200326.20201005-S230611-E003844.037527.V06X.HDF5data/2A.GPM.DPRX.V8-20200326.20201005-S060804-E074036.037516.V06X.HDF5
data/2A.GPM.DPRX.V8-20200326.20201005-S074037-E091309.037517.V06X.HDF5
data/2A.GPM.DPRX.V8-20200326.20201005-S091310-E104543.037518.V06X.HDF5
data/2A.GPM.DPRX.V8-20200326.20201005-S104544-E121816.037519.V06X.HDF5



In [2]:
combine_ds

<xarray.Dataset>
Dimensions:  (along_track: 39667, cross_track: 49, range: 176)
Coordinates:
    lons     (along_track, cross_track) float32 44.24 44.24 ... -151.8 -151.8
    lats     (along_track, cross_track) float32 -66.27 -66.22 ... -64.04 -63.98
    time     (along_track, cross_track) datetime64[ns] 2020-10-05T23:06:12 .....
    alt      (along_track, cross_track, range) float64 20.86 20.74 ... 0.1191
Dimensions without coordinates: along_track, cross_track, range
Data variables:
    NSKu_c   (along_track, cross_track, range) float32 nan nan nan ... nan nan